In [19]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
import datetime
import psycopg2
from sqlalchemy import create_engine
import sqlalchemy
import matplotlib.pyplot as plt
import seaborn as sns

In [20]:
dropbox_dir = r'/Users/kendraziegler/Dropbox (Palmer Lab)/Palmer Lab/Shae Ziegler/Suzanne_Mitchell_U01'
df_fits = pd.read_csv(dropbox_dir + '/fits.csv')
df_locomotor = pd.read_csv(dropbox_dir + '/locomotor.csv')
df_wfu = pd.read_csv(dropbox_dir + '/wfu_master.csv')
df_fecal_boli = pd.read_csv(dropbox_dir + '/fecal_boli.csv', dtype = str)

In [21]:
df_wfu

,cohort,sires,dames,labanimalid,accessid,sex,rfid,dob,dow,shipmentdate,...,coatcolor,earpunch,rack,shipmentbox,housingbox,shipmentage,weanage,comments,resolution,last_5
0,C01,72539_1,72595_5,TJ008,73472_2,F,933000320045906,2018-09-24,2018-10-15,2018-10-30,...,BROWNHOOD,RB,D-F4,2.0,9999.0,36.0,21.0,Pregnant female/Impregnated,REMOVE_FROM_EXCLUSION_AND_REPLACE,45906
1,C01,73358_1,73360_1,TJ027,73477_2,F,933000320045890,2018-09-25,2018-10-16,2018-10-30,...,BROWNHOOD,LM,D-G8,4.0,32.0,35.0,21.0,NaN,NaN,45890
2,C01,72539_1,72595_5,TJ003,73472_8,M,933000320045902,2018-09-24,2018-10-15,2018-10-30,...,ALBINO,RT,D-F2,2.0,9999.0,36.0,21.0,Pregnant female/Impregnated,REMOVE_FROM_EXCLUSION_AND_REPLACE,45902
3,C01,72794_1,72775_4,TJ020,73475_4,F,933000320045904,2018-09-25,2018-10-16,2018-10-30,...,BROWN,RB,D-G2,2.0,9999.0,35.0,21.0,Pregnant female/Impregnated,REMOVE_FROM_EXCLUSION_AND_REPLACE,45904
4,C01,72624_1,72596_4,TJ002,73471_5,F,933000320045908,2018-09-23,2018-10-15,2018-10-30,...,BLACK,LT,D-F2,2.0,9999.0,37.0,22.0,Pregnant female/Impregnated,REMOVE_FROM_EXCLUSION_AND_REPLACE,45908
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
896,C07,933000320048190,933000320187377,NaN,NaN,F,933000320125436,2020-08-09,2020-09-04,NaN,...,BLACK,NaN,NaN,NaN,NaN,NaN,26.0,NaN,NaN,25436
897,C07,933000320187153,933000320187383,NaN,NaN,F,933000320125437,2020-08-09,2020-09-04,NaN,...,BROWNHOOD,NaN,NaN,NaN,NaN,NaN,26.0,NaN,NaN,25437
898,C07,933000320187217,933000320048245,NaN,NaN,F,933000320125440,2020-08-09,2020-09-04,NaN,...,BROWN,NaN,NaN,NaN,NaN,NaN,26.0,NaN,NaN,25440
899,C07,933000320187249,933000320048234,NaN,NaN,M,933000320125443,2020-08-08,2020-09-04,NaN,...,BLACKHOOD,NaN,NaN,NaN,NaN,NaN,27.0,NaN,NaN,25443


In [22]:
def dd_fits(df_fits):
    df_fits = df_fits.add_prefix('dd_')
    df_fits.rename(columns={'dd_sex':'sex','dd_cohort':'cohort','dd_rfid':'rfid'},inplace=True)
    df_fits.drop(['sex'],axis = 1, inplace = True)

    return df_fits
df_fits = dd_fits(df_fits)
#list(df_fits.columns.values)
#df_fits['cohort'].value_counts()


In [23]:
#find all rfid that are in dd_fits or locomotor
rfid_fits = list(df_fits['rfid'])
rfid_locomotor = list(df_locomotor['rfid'])
wfu_needed = list(set(rfid_fits + rfid_locomotor))
wfu_needed = [str(x) for x in wfu_needed]
#wfu_needed

In [24]:
def locomotor(df_locomotor):
    df_locomotor.drop(['sample'],axis = 1, inplace = True)
    return df_locomotor
        
        
df_locomotor = locomotor(df_locomotor)        
#list(df_locomotor.columns.values)
#df_locomotor['cohort'].value_counts()


In [25]:
def fecal_boli():
    df_fecal_boli.rename(columns = {'box':'fecal_box','date_1':'fecal_date_1','time_in_1':'fecal_time_in_1'
                                    ,'date_2':'fecal_date_2','time_in_2':'fecal_time_in_2',
                                    'change':'fecal_change'},inplace=True)
    df_fecal_boli.drop(['sex','last_5'],axis = 1, inplace = True)

    
fecal_boli()
#df_fecal_boli['cohort'].value_counts()
#list(df_fecal_boli.columns.values)


In [26]:
def wfu_columns():
    df_wfu.drop(['last_5'],axis = 1, inplace = True)

wfu_columns()

In [27]:
#Changes the int rfid and last_5 columns into strings
def reformat(df, columns):
    for col in columns:
        df[col] = df[col].astype(str)
        df[col] = [val.replace('.0','') for val in df[col]]
        
reformat(df_fecal_boli, ["rfid"])
reformat(df_fits, ["rfid"])
reformat(df_locomotor, ["rfid"])
reformat(df_wfu, ["rfid","housingbox"])


In [28]:
len(wfu_needed)

500

In [29]:
df_fits


,cohort,rfid,dd_coat_color,dd_shipping_box,dd_housing_box,dd_order_in_box,dd_squad_num,dd_box_color,dd_opertant_box,dd_id_check,...,dd_indiff_8,dd_indiff_16,dd_indiff_24,dd_hyperbolic_k,dd_hyperbolic_lnk,dd_hyperbolic_b,dd_AUC_traditional,dd_AUC_normal,dd_quasi_H_k_beta,dd_quasi_H_s_delta
0,C02,933000320045751,brown_hood,5,40,2,4,blue,8,0,...,49.205815,22.286333,30.258215,0.152786,-1.878717,0.794732,0.319148,0.401579,0.815571,0.905189
1,C02,933000320045756,black,11,54,2,5,blue,12,0,...,62.431850,37.152422,31.230381,0.083817,-2.479118,0.640505,0.353315,0.551620,0.595001,0.952806
2,C02,933000320045757,brown_hood,11,18,2,2,blue,12,0,...,43.284175,42.058929,29.908862,0.249252,-1.389293,0.796148,0.312850,0.392955,0.409958,0.971604
3,C02,933000320045759,brown_hood,10,52,2,5,blue,8,0,...,69.614073,31.810227,26.974896,0.151994,-1.883917,1.068606,0.431175,0.403493,0.987392,0.921055
4,C02,933000320045760,brown,5,39,2,4,blue,6,0,...,90.602227,45.685779,31.550220,0.172621,-1.756656,1.229121,0.498878,0.405882,1.008066,0.931037
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,C04,933000320047963,black_hood,1,12,2,1,red,8,0,...,59.189177,53.073730,NaN,0.071768,-2.634312,0.998539,NaN,NaN,2.027855,0.831496
391,C04,933000320047964,black_hood,6,34,1,3,red,3,0,...,84.682184,66.694060,36.386721,0.007513,-4.891068,0.523546,0.576590,1.101316,1.291869,0.928345
392,C04,933000320047968,brown_hood,1,27,1,3,blue,5,0,...,116.862215,58.987025,46.116756,0.155155,-1.863329,1.378207,0.606703,0.440212,1.037585,0.948342
393,C04,933000320047973,black,1,12,1,1,red,7,0,...,113.984993,64.902099,37.109793,0.087389,-2.437385,1.134337,0.616921,0.543860,1.102845,0.945492


In [30]:
df_fits_locomotor = pd.merge(df_wfu, df_locomotor, on=['rfid','cohort'],how = 'outer')
df_wfu_fecal = pd.merge(df_fits_locomotor, df_fecal_boli, on=['rfid','cohort'],how = 'outer')
df_all = pd.merge(df_wfu_fecal, df_fits, on=['rfid','cohort'],how = 'outer')

df_all_final = df_all.loc[df_all['rfid'].isin(wfu_needed)]
df_all_final
#df_all_final['cohort'].value_counts()


,cohort,sires,dames,labanimalid,accessid,sex,rfid,dob,dow,shipmentdate,...,dd_indiff_8,dd_indiff_16,dd_indiff_24,dd_hyperbolic_k,dd_hyperbolic_lnk,dd_hyperbolic_b,dd_AUC_traditional,dd_AUC_normal,dd_quasi_H_k_beta,dd_quasi_H_s_delta
1,C01,73358_1,73360_1,TJ027,73477_2,F,933000320045890,2018-09-25,2018-10-16,2018-10-30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,C04,73732_1,73686_2,MIT313,74790_1,F,933000320047539,2019-08-31,2019-09-23,2019-10-08,...,113.076852,86.788228,38.608136,0.028627,-3.553404,0.740578,0.603052,0.814299,0.807353,0.969964
12,C04,74217_3,74208_2,MIT363,74794_2,M,933000320047540,2019-09-01,2019-09-23,2019-10-08,...,127.939277,71.965546,37.166737,0.082575,-2.494043,1.521417,0.808779,0.531596,2.197358,0.900974
13,C05,74862_3,74718_1,MIT426,75332_7,M,933000320187236,2019-12-22,2020-01-13,2020-02-04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,C05,74228_1,74195_2,MIT417,75336_1,M,933000320186971,2019-12-22,2020-01-13,2020-02-04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
505,C04,74147_1,74217_4,MIT322,74763_4,F,933000320047974,2019-08-25,2019-09-16,2019-10-08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
506,C04,73740_1,73747_2,MIT319,74759_1,M,933000320047975,2019-08-25,2019-09-16,2019-10-08,...,71.246033,47.373087,29.742643,0.149225,-1.902303,1.076422,0.461549,0.428781,0.911313,0.934968
507,C04,73690_1,73701_2,MIT386,74799_2,F,933000320047698,2019-09-05,2019-09-26,2019-10-08,...,77.603314,52.888306,48.197887,0.169942,-1.772300,1.197080,0.501697,0.419101,0.896912,0.945514
508,C04,74194_1,74167_2,MIT347,74783_2,M,933000320047544,2019-08-29,2019-09-19,2019-10-08,...,66.674486,50.233641,25.935215,0.141524,-1.955285,1.013645,0.447781,0.441753,0.879342,0.935071


In [61]:
def data_present():
    #Check to see how many rows are all nan True means not all are nan, false means all are nan
    nan_list_loco = list(df_all_final.iloc[:, [23,39]].notnull().any(1))
    nan_values_loco = [[x,nan_list_loco.count(x)] for x in set(nan_list_loco)]
    #print(nan_values_loco)
    #Creates new column based on whether or not all locomotor columns contain nan
    df_all_final['locomotor_data_present'] = pd.notnull(df_all_final.iloc[:, [23,39]]).any(axis=1)
    
    #repeat for dd traits
    nan_list_dd = list(df_all_final.iloc[:, [58,70]].isna().any(1))
    nan_values_dd = [[x,nan_list_dd.count(x)] for x in set(nan_list_dd)]
    #print(nan_values_dd)
    #Creates new column based on whether or not all dd columns contain nan
    df_all_final['dd_data_present'] = pd.notnull(df_all_final.iloc[:, [58,70]]).any(axis=1)
        
data_present()
df_all_final['dd_data_present'].value_counts()


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


True     395
False    105
Name: dd_data_present, dtype: int64

In [32]:
df_all_final['cohort'].value_counts()
list(df_all_final.columns.values)

['cohort',
 'sires',
 'dames',
 'labanimalid',
 'accessid',
 'sex',
 'rfid',
 'dob',
 'dow',
 'shipmentdate',
 'litternumber',
 'littersize',
 'coatcolor',
 'earpunch',
 'rack',
 'shipmentbox',
 'housingbox',
 'shipmentage',
 'weanage',
 'comments',
 'resolution',
 'cage_x',
 'locomotor_t1_experiment',
 'locomotor_t1_time',
 'locomotor_t1_file_num',
 'locomotor_t2_time',
 'locomotor_t2_file_num',
 'locomotor_t2_experiment',
 'locomotor_t1_total_distance',
 'locomotor_t1_rest_time',
 'locomotor_t1_rest_episode_count',
 'locomotor_t1_movement_episode_count',
 'locomotor_t1_vertical_activity_count',
 'locomotor_t1_center_time_legacy',
 'locomotor_t2_total_distance',
 'locomotor_t2_rest_time',
 'locomotor_t2_rest_episode_count',
 'locomotor_t2_movement_episode_count',
 'locomotor_t2_vertical_activity_count',
 'locomotor_t2_center_time_legacy',
 'fecal_box',
 'fecal_date_1',
 'fecal_time_in_1',
 'fecal_boli_1',
 'fecal_date_2',
 'fecal_time_in_2',
 'fecal_boli_2',
 'fecal_change',
 'dd_coat

In [40]:
df_all_final.columns.get_loc('dd_quasi_H_s_delta')

70